In [ ]:
import pandas as pd
import os
import zipfile
import requests

# Downloading  CSV file from Google Drive
def download_file_from_google_drive(url, output_path):
    session = requests.Session()
    response = session.get(url, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'confirm': token}
        response = session.get(url, params=params, stream=True)

    save_response_content(response, output_path)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, output_path):
    CHUNK_SIZE = 32768

    with open(output_path, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

download_url = "https://drive.google.com/file/d/1poeJoZ7E0B-6Q2PYd_n9MIGlPXCfX7Kn/view?usp=sharing"
destination = 'sf-salary-data.csv'
download_file_from_google_drive(download_url, destination)

# Loading the downloaded CSV file
df = pd.read_csv(destination)

def get_employee_details(name):
    try:
        employee = df[df['EmployeeName'].str.contains(name, case=False, na=False)]
        if not employee.empty:
            return employee.to_dict(orient='records')
        else:
            return "Employee not found."
    except Exception as e:
        return f"An error occurred: {e}"

# Creating a dictionary with employee names as keys
employee_dict = df.set_index('EmployeeName').T.to_dict('list')

# Example access
name = "NATHANIEL FORD"
details = employee_dict.get(name, "Employee not found")

def export_employee_details(name, output_dir="Employee_Profile"):
    try:
        details = get_employee_details(name)
        if isinstance(details, list):
            # Create directory if not exists
            os.makedirs(output_dir, exist_ok=True)
            
            # Export details to CSV
            file_path = os.path.join(output_dir, f"{name.replace(' ', '_')}_details.csv")
            pd.DataFrame(details).to_csv(file_path, index=False)
            
            # Zip the folder
            zip_file = f"{output_dir}.zip"
            with zipfile.ZipFile(zip_file, 'w') as zipf:
                for root, dirs, files in os.walk(output_dir):
                    for file in files:
                        zipf.write(os.path.join(root, file))
            return f"Exported and zipped to {zip_file}"
        else:
            return details
    except Exception as e:
        return f"An error occurred: {e}"

# Printing Details of Nathaniel Ford
print(export_employee_details("NATHANIEL FORD"))
